In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from ipywidgets import widgets, interactive, interact, Layout, Box, HTML as ipyHTML
from IPython.display import display, HTML, clear_output

df_static_paths = pd.read_pickle("df_static_paths.pkl")
df_dynamic_paths = pd.read_pickle("df_dynamic_paths.pkl")
metrics_df = pd.read_pickle("metrics_df.pkl")
df_p = pd.read_pickle("portfolio.pkl")
df_tp = pd.read_pickle("target_portfolio.pkl")

In [ ]:
# %%html
# <style>
# .output_wrapper, .output {
#     height:auto !important;
# }
# .output_scroll {
#     box-shadow:none !important;
#     webkit-box-shadow:none !important;
# }
# .output_subarea table {
#     box-shadow: 10px 5px 5px black;
#     border: 4px solid rgba(0,0,0,0.8);
# }
# #.widget-datepicker {
#     #visibility: hidden;
# #}
# </style>

In [21]:
%%html
<style>
.widget-select-multiple option {
    font-size: 26px;
    margin: 5px 0px 10px;
}
.widget-select-multiple select {
    padding: 0px 20px;
}
div.output_scroll {
    overflow: scroll;
    height: 60em;
}
.output_scroll > .output_area:first-child {
    border-bottom: 3px solid black;
    min-height: 30px;
}
.output_scroll > .output_area:nth-child(2) {
    overflow: auto;
}
.output_scroll > .output_area:nth-child(2) .output_subarea {
    overflow: scroll;
    min-width: 1015px;
}

</style>

In [4]:
batchid = widgets.Select(options=[917239704889096439], description="Select batchid")

### Batch View

In [13]:
def parameter_view(batchid, df_static_paths, df_dynamic_paths):
    template = """<H2>batchid: {}</H2><br> Run: {}<br><h3>Static Parameters</h3>"""
    display(HTML(template.format(batchid, "2017-06-11")))
    display(df_static_paths, height = 400)
    display(HTML("<h3>Varied Parameters</h3>"))
    display(df_dynamic_paths, height=400)

In [17]:
def batchid_summary_view(batchid, df_dynamic_paths):
#     metrics_df = df_dynamic_paths.reset_index().rename(
#         columns={'index': 'simulationid'}
#     ).merge(metrics_df.drop('created_at', axis=1), how='left', on='simulationid')

    """setting html style for metrics dataframe"""
    styles =[
        {'selector': '.row_heading, .blank', 'props': [('display', 'none;')]},
        {'selector': 'tr:nth-child(even)', 'props': [('background-color', '#f2f2f2;')]},
        {'selector': 'td', 'props': [('height', '20px;')]},
    ]
    attributes = 'class="table";'

    """multiselect box for plots"""
    plotboxes = widgets.SelectMultiple(
        options=[x for x in metrics_df.index],        
        disabled=False,
        layout=Layout(display="flex", flex_flow='column', height='100%', width='100%')
    )

    left_box = widgets.VBox([widgets.Label(value = 'Plot ?'), plotboxes]) 
    global table
    table = widgets.HBox([
        left_box, 
        ipyHTML(
            metrics_df.style.set_table_styles(styles).set_table_attributes(attributes).render(),
            layout=Layout(overflow_x='scroll', hight = '50%')
               )
    ])

    display(table)
pd.set_option('display.max_colwidth', 3000)
pd.set_option('display.precision', 12) 

In [18]:
def interact_combined_view(batchid):
    """This is the plot function for display both parameter_view and batchid_summary_view"""
    clear_output(True)

    parameter_view(batchid, df_static_paths, df_dynamic_paths)
    batchid_summary_view(batchid, df_dynamic_paths)
    

pd.set_option('display.max_colwidth', 3000)
pd.set_option('display.precision', 12)    

"""main container"""
interactive(
    interact_combined_view, 
    batchid=batchid
)

### Portfolio View

In [ ]:
def portfolio_view(_df_p):
    """This funtion returns a new view of portfolio dataframe and keep track of net_exposure of portfolio at date"""
    df = _df_p.copy()

    df['value'] = (df.price * df.shares)
    df_net_exposure = df.groupby(by=['date'])[['value']].sum().rename(columns={'value': 'net_exposure'}).reset_index()
    df = pd.merge(df, df_net_exposure, on='date', how='outer')    
    df['percent_of_net'] = (df.value / df.net_exposure)
    df['amount'] = (df.price * df.shares.abs())
    df['extra_stuff1'] = 'TBD1'
    df['extra_stuff2'] = 'TBD2'
    df['extra_stuff3'] = 'TBD3'
    df['extra_stuff4'] = 'TBD4'
    
    """select cols"""
    cols = ['date', 'tradingitemid', 'companyname', 'symbol', 'exchange', 'price', 'shares', 
            'value', 'percent_of_net', 'batch', 'name', 'amount', 'extra_stuff1', 'extra_stuff2', 'extra_stuff3',
           'extra_stuff4']
    df = df.loc[:, cols]
    
    return df, df_net_exposure

df_p_view, df_net_exposure = portfolio_view(df_p)    


def portfolio_view_date_select(date):
    """This function interacts with datePick and SortBy to display the target_portfolio dataframe"""
    df = df_p_view.loc[df_p_view.date == date, :]  
    
    print("net exposure: " + str(df_net_exposure.loc[df_net_exposure.date == date].iloc[0,1]))

    display(df)

    

"""create the interactive object""" 
interact_df_p = interactive(portfolio_view_date_select, 
                            date=widgets.DatePicker(value = df_p_view.date[0], description='Date'))

"""button for skip date"""
last_week = widgets.Button(description="previous date") 
next_week = widgets.Button(description="next date")    

df_date_t = pd.DataFrame({"date": df_p.date.unique()})

def forward(next_week):
    interact_df_p.children[0].value=df_date_t.iloc[df_date_t.index[df_date_t.date==interact_df_p.children[0].value] + 1].iloc[0,0]
    
def backward(last_week):
    interact_df_p.children[0].value=df_date_t.iloc[df_date_t.index[df_date_t.date==interact_df_p.children[0].value] - 1].iloc[0,0]

next_week.on_click(forward)
last_week.on_click(backward)  

button_box = widgets.HBox([widgets.Label("Click here: "), last_week, next_week])    


display(button_box, interact_df_p)

pd.set_option("display.max_rows", None)

In [ ]:
display(HTML("<h3>Target Portfolio View</h3>"))

In [20]:
"""This chunk is for displaying df_target_portfolio"""
def target_portfolio_view(_df_tp):
    """This funtion takes target_portfolio dataframe and returns a new view of relevant feilds sorted by date"""     
    df = _df_tp.copy()
    """cols to display"""
    cols = ['date', 'tradingitemid', 'companyname', 'symbol', 'exchange', 'score', 'position_type',
            'amount', 'price', 'limit_lo', 'limit_hi']    
    return df.loc[:, cols]

"""store the df_tp in df_tp_view"""
df_tp_view = target_portfolio_view(df_tp)

def target_portfolio_view_date_select(date, sortby, ascending, filterby, tradingitemid):
    """This function interacts with datePick and SortBy to display the target_portfolio dataframe"""
    """sort first"""
    if not filterby:        
        df = df_tp_view.loc[df_tp_view.tradingitemid == tradingitemid, :].sort_values(by=sortby, ascending=ascending)
    else:
        df = df_tp_view.loc[
            df_tp_view.date == date, :
        ].loc[df_tp_view.position_type != filterby, :].sort_values(by=sortby, ascending=ascending)   
    """format"""    
    df['tradingitemid'] = df['tradingitemid'].astype(int)
    df['amount'] = df['amount'].map('{:,.2%}'.format)
    df['limit_hi'] = df['limit_hi'].map('${:,.2f}'.format) 
    df['limit_lo'] = df['limit_lo'].map('${:,.2f}'.format)  
    df['price'] = df['price'].map('${:,.2f}'.format) 
    df['score'] = df['score'].map('{:,.2%}'.format)
    ##############################   
    print ("place_holder")
    pd.set_option("display.max_rows", None)
    display(df)
 
"""create the interactive object"""
if len(df_tp_view) == 0:
    interact_df_tp = None
    print('I am sorry, data is empty')
    
else:
    interact_df_tp = interactive(target_portfolio_view_date_select, 
                             date= widgets.DatePicker(value = df_tp_view.date[0], description='Date'),
                                                      #layout=Layout(height='180px')),
                             sortby = widgets.Dropdown(options=[col for col in df_tp_view.columns.values],
                                                       value=df_tp_view.columns.values[0],
                                                       description='Sort by'),
                                                       #layout=Layout(height='250px')) 
                             ascending = widgets.Dropdown(options={"Ascending": True, "Descending": False},
                                                          value=True,
                                                          description='Sort order'),
                             filterby = widgets.Dropdown(options={"All": "all", "Only Long": "SHORT_EQUITY", 
                                                                  "Only Short": "LONG_EQUITY", "tradingitemid": False},
                                                         value="all",
                                                         description='Filter by'),
                             tradingitemid = widgets.FloatText(description='Filter by tradingitemid ', 
                                                               value=df_tp_view.tradingitemid[0])
                            )
"""button for skip date"""
last_month = widgets.Button(description="previous date") 
next_month = widgets.Button(description="next date")    

df_date_tp = pd.DataFrame({"date": df_tp_view.date.unique()})

def forward(next_month):    
    interact_df_tp.children[0].value=df_date_tp.iloc[df_date_tp.index[df_date_tp.date==interact_df_tp.children[0].value] + 1].iloc[0,0]
    
    
def backward(last_month):
    interact_df_tp.children[0].value=df_date_tp.iloc[df_date_tp.index[df_date_tp.date==interact_df_tp.children[0].value] - 1].iloc[0,0]

next_month.on_click(forward)
last_month.on_click(backward)    
    
"""box"""    
button_box = widgets.HBox([widgets.Label("Click here: "), last_month, next_month])      
"""main container display"""
display(button_box, interact_df_tp)

# pd.set_option("display.max_rows", None)